<a href="https://colab.research.google.com/github/medikid/ai_lotto/blob/master/notebooks/git_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Convert IPyNB to Py

In [1]:
#Save all ipynb files to py and make sure first line has %%WRITEFILE to py files

%run 'ifile.ipynb'

%run 'idata.ipynb'
%run 'idataset.ipynb'

%run 'imodel.ipynb'
%run 'ker_model.ipynb'
%run 'ker_model_loader.ipynb'
%run 'tfl_model.ipynb'


%run 'ai.ipynb'
%run 'ipredictor.ipynb'
%run 'itrainer.ipynb'

%run 'custom_functions.ipynb'

Overwriting ../ifile.py
Overwriting ../idata.py
Overwriting ../idataset.py
Overwriting ../imodel.py
Overwriting ../ker_model.py
Overwriting ../ker_model_loader.py
Overwriting ../tfl_model.py
Overwriting ../ai.py
Overwriting ../ipredictor.py


Using TensorFlow backend.


Setting up sqlite  sqlite:////home/work/ai_lotto/notebooks/db/ai_lotto.db
Setting up sqlite  sqlite:////home/work/ai_lotto/notebooks/db/ai_lotto.db
xndraw2.528m, new_st2.528, multiple1000000, notationm
[iFile:__init__]
xndraw2.528m, new_st2.528, multiple1000000, notationm
[iFile:derive_game_path]data/keno/
[iFile:derive_folder_path] data/keno/x15/2.528m/
[iDataset:derive_file_path] datasets/keno_x15_2.528m_dr.npz
[ifile:derive_full_path] ../data/keno/x15/2.528m/datasets/keno_x15_2.528m_dr.npz
Loading dataset ../data/keno/x15/2.528m/datasets/keno_x15_2.528m_dr.npz
[iModel:decipher_file_name]
[iModel:set_decipher_info] {'GAME': 'keno', 'xnINPUTS': 'x15', 'nINPUTS': 15, 'xnDRAWS': '2.528m', 'nDRAWS': 2528000, 'DATA_TYPE': 'dr', 'API': 'ker', 'BUILD': '4', 'MAKE': '2', 'VERSION': 'e0010', 'IS_CHECKPOINT': True, 'CHECKPOINT_EPOCH': 10}
[iFile:__init__]
[iModel:decipher_file_name]
[iModel:set_decipher_info] {'GAME': 'keno', 'xnINPUTS': 'x15', 'nINPUTS': 15, 'xnDRAWS': '2.528m', 'nDRAWS': 252

## Run Py files to memory

In [1]:
# %load '../load_lib.py'
# then import lib as 'from ifile import iFile' on your notebooks

%run '../ifile.py'
%run '../idata.py'
%run '../idataset.py'
%run '../imodel.py'
%run '../ker_model.py'
%run '../ker_model_loader.py'
%run '../tfl_model.py'
%run '../ai.py'
%run '../itrainer.py'
%run '../custom_functions.py'
%run '../custom_callbacks.py' #is created from second cell on custom functions

Using TensorFlow backend.


# GIthub Functions

## Git ADD, COMMIT and PUSH to remote

In [3]:
!git add -A .
!git commit -m "fixed csv log file location"
!git push origin master

[master 644211e] fixed csv log file location
 2 files changed, 21 insertions(+), 10 deletions(-)
Counting objects: 5, done.
Delta compression using up to 6 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 895 bytes | 447.00 KiB/s, done.
Total 5 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To git+ssh://github.com/medikid/ai_lotto.git
   ceee740..644211e  master -> master


## Git Pull from remote github repo

In [5]:
!git pull

Already up to date.


## Git conflict local - so reset local to match remote

In [ ]:
!git fetch --all
!git reset --hard origin/master
!git pull

## Unpushed local changes - Stash local, Pull Remote and write local changes to pulled remote

In [ ]:
!git stash
!git pull
!git stash pop

## Setup git config repo

In [ ]:
#!git config --global user.email "soma.programmer@gmail.com"
#!git config --global user.name "medikid"

#!git remote set-url origin https://github.com/medikid/ai_lotto.git


In [ ]:
!git remote show origin

# Sync G-Drive with Data Folder

In [ ]:
#!pip install dirsync

from dirsync import sync
source_path = '../g_drive'
target_path = '../data'

sync(source_path, target_path, 'sync', verbose  = True) #for syncing one way
sync(target_path, source_path, 'sync', verbose = True) #for syncing the opposite way

# sync(source_path, target_path, 'diff') #for syncing one way
# sync(target_path, source_path, 'diff') #for syncing the opposite way

In [ ]:
!tree '../data/'

In [ ]:
import os, shutil

#shutil.copy(a,b)
src_folder = "../data"
dst_folder = "../archive"
for src_file_path in [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("../data")) for f in fn]:
    src_folder_path = os.path.dirname(src_file_path)
    src_file_format = src_file_path.split('.')[-1]; # 'h5'
    src_file_name = os.path.basename(src_file_path);
    game = st.split('/')[2]
#     print("Full path: {0} \n \
#     folder_path: {1} \n \
#     file_name: {2} \n \
#     file_format: {3}".format(full_path, folder_path, file_name, file_format ));
    if (src_file_format == 'h5'):
        dst_file_name = '[{0}]{1}'.format(src_dataset_id, src_file_name);
        dst_file_path = '{0}/{1}/models/{2}.{3}'.format(dst_folder, game, dst_file_name,src_file_format)

        
    

In [ ]:
st = '../data/keno/master/keno_master_dr'
#st.split('.')[-1]
st.split('/')[2]

## Archive files into archive folder

In [7]:
%%writefile '../archiver.py'

import os, shutil

class Archiver:
    srcFolder = "../data"
    dstFolder = "../archive"
    srcFiles = []
    dstFiles = []
    
    def __init__(self, src='../data', dst='../archive'):
        self.setup(src, dst);
        self.process_files();
        
        self.setup('../g_drive', dst);
        self.process_files();
        
    def setup(self, src, dst):
        self.srcFolder = src;
        self.dstFolder = dst;
        self.srcFiles = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(self.srcFolder)) for f in fn];
        self.dstFiles = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(self.dstFolder)) for f in fn];
        print('Source:{0} Files'.format(len(self.srcFiles)))
        print('Dest: {0} Files'.format(len(self.dstFiles)))
        
        
    def process_files(self):        
        for srcFilePath in self.srcFiles:
            srcFolderPath = os.path.dirname(srcFilePath)
            srcFileFormat = srcFilePath.split('.')[-1]; # 'h5'
            srcFileName = os.path.basename(srcFilePath);
            game = srcFilePath.split('/')[2]
            if (srcFileFormat == 'h5' or srcFileFormat == 'ckpt' or srcFileFormat[0] == 'e'): 
                pass;
                self.archive_h5_ckpt_e(srcFilePath);
            elif (srcFileFormat == 'pkl'):
                pass;
                self.archive_pkl(srcFilePath);
            elif (srcFileFormat == 'npz'):
                self.archive_npz(srcFilePath);
                print('FileName:{0}[{1} MB]'.format(srcFilePath,round(os.path.getsize(srcFilePath)/(1024*1024)),2) )
            else:
                print("Unknown format: {0}".format(srcFilePath))
                    
    def archive_h5_ckpt_e(self, srcFilePath):
        srcFolderPath = os.path.dirname(srcFilePath)
        srcFileName = os.path.basename(srcFilePath);
        srcFileFormat = srcFilePath.split('.')[-1];
        folders = srcFolderPath.split('/');
        try:
            game = folders[2];
            xnInputs = folders[3];
            xnDraws = folders[4]
            srcDatasetID = '{0}_{1}_{2}_dr'.format(game,xnInputs,xnDraws)
            dstFileName = '{0}[{1}]'.format(srcFileName, srcDatasetID,);
            dstFilePath = '{0}/{1}/models/{2}'.format(self.dstFolder, game, dstFileName)
            #print('{0}=>{1}'.format(srcFilePath, dstFilePath))
            if(os.path.exists(dstFilePath)):
                pass;
            else:
                self.archive(srcFilePath, dstFilePath);
            
        except IndexError:
            if (folders[3] == 'untrained_models'):
                game = folders[2]
                dstFileName = '[untrained]{0}'.format(srcFileName);
                dstFilePath = '{0}/{1}/models/{2}'.format(self.dstFolder, game, dstFileName)
                if(os.path.exists(dstFilePath)):
                    pass;
                else:
                    self.archive(srcFilePath, dstFilePath);
            else:
                print("exception",srcFilePath)
    
    def archive_pkl(self, srcFilePath):
        game = srcFilePath.split('/')[2]
        isMaster = srcFilePath.split('/')[3]
        srcFileName = os.path.basename(srcFilePath);
        dstFileName = srcFileName;
        if (isMaster == 'master'):
            dstFilePath = '{0}/{1}/masters/{2}'.format(self.dstFolder, game, dstFileName)
        
        if(os.path.exists(dstFilePath)):
                pass;
        else:
            self.archive(srcFilePath, dstFilePath);
            
    def archive_npz(self, srcFilePath):
        game = srcFilePath.split('/')[2]
        srcFileName = os.path.basename(srcFilePath);
        dstFileName = srcFileName;
        dstFilePath = '{0}/{1}/datasets/{2}'.format(self.dstFolder, game, dstFileName)
        #print(srcFilePath)
        if(os.path.exists(dstFilePath)):
                pass;
        else:
            self.archive(srcFilePath, dstFilePath);
            
    def archive(self, srcFilePath, dstFilePath):
        shutil.copy(srcFilePath, dstFilePath);
        print("Copying {0}".format(os.path.basename(dstFilePath)));
    

Overwriting ../archiver.py


In [ ]:
%run '../archiver.py'
arc = Archiver('../data');

Source:1908 Files
Dest: 3455 Files
Unknown format: ../data/print_folder_tree.ipynb
FileName:../data/hotspot/x15/25k/datasets/hotspot_x15_25k_dp.npz[141 MB]
FileName:../data/hotspot/x15/25k/datasets/hotspot_x15_25k_dP.npz[0 MB]
FileName:../data/hotspot/x15/25k/datasets/hotspot_x15_25k_dr.npz[244 MB]
Copying hotspot.ker.4.13.e0490.h5[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0122.ckpt[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0169.ckpt[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0435.ckpt[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0198[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0438.h5[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0095[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0101[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0176[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0467.h5[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0505.h5[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0006[hotspot_x15_25k_dr]
Copying hotspot.ker.4.13.e0350.h5[hotspot_x15_25k_dr]
Copying

# Print folder tree

In [10]:
#!tree '../archive/keno'
!tree '../data/hotspot/x15'

../data/hotspot/x15
├── 100k
│   └── datasets
│       ├── hotspot_x15_100k_dp.npz
│       └── hotspot_x15_100k_dr.npz
├── 200k
│   └── datasets
│       ├── hotspot_x15_200k_dp.npz
│       └── hotspot_x15_200k_dr.npz
├── 25k
│   ├── datasets
│   │   ├── hotspot_x15_25k_dp.npz
│   │   ├── hotspot_x15_25k_dP.npz
│   │   └── hotspot_x15_25k_dr.npz
│   └── models
│       └── ker
│           ├── 1
│           │   └── 1
│           ├── 2
│           │   └── 1
│           ├── 3
│           │   └── 1
│           ├── 4
│           │   ├── 1
│           │   ├── 13
│           │   │   └── checkpoints
│           │   │       ├── hotspot.ker.4.13.e0001.h5
│           │   │       ├── hotspot.ker.4.13.e0002
│           │   │       ├── hotspot.ker.4.13.e0002.ckpt
│           │   │       ├── hotspot.ker.4.13.e0002.h5
│           │   │       ├── hotspot.ker.4.13.e0003
│           │   │       ├── hotspot.ker.4.13.e0003.ckpt
│           │   │       ├── hotspot.ker.4.13.e0003.h5
│           │   │       ├── 

In [22]:
# FileRenamer

import os, shutil

class FileRenamer:
    srcFolder = "../data/"
    srcFiles = []
    checkpointFiles = []
    
    def __init__(self, fileFormat='.h5'):
        self.setup();
        pass
    
    def setup(self):
        self.srcFiles = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(self.srcFolder)) for f in fn];
    
    def renameCheckpointFiles(self):
        #file checkpoint files and then add _Version#0 e.g. hotspot.ker.4.13.e0001 to hotspot.ker.4.13.0.e0001
        for file in self.srcFiles:
            folderName = file.split('/')[-2];
            if (folderName == 'checkpoints'):
                renameFileTo = file.replace('.e0','.0.e0');
                os.rename(file, renameFileTo)
                print("Changed to {0}".format(renameFileTo))

In [23]:
frname = FileRenamer();
frname.setup();
frname.renameCheckpointFiles()

Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0122.ckpt
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0169.ckpt
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0435.ckpt
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0198
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0438.h5
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0095
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0101
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0176
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0467.h5
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0505.h5
Changed to ../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.e0006
Changed 

In [24]:
fn = '../data/hotspot/x15/25k/models/ker/4/13/checkpoints/hotspot.ker.4.13.0.0.e0490.h5'
fnc = fn.replace('.0.e0','.e0')
fnc
import os
os.rename(fn, fnc)